In [ ]:
/* ===================================================================================
   METRIC: EMP03 - Timely Training Completion
   
   BUSINESS QUESTION: 
   What is the timely completion percentage for the Code of Conduct and Ethics 
   training within each Assessable Unit?
   
   LOGIC SUMMARY:
   1. Evaluates each employee's Training Record.
   2. If the Completion Date is on or before the Due Date, they score 1 (On Time).
   3. If the Completion Date is late, blank, or the Due Date is missing, they score 0.
   4. Rolls up by Assessable Unit and calculates: 
      (Total Completed On Time / Total Employees) * 100.
=================================================================================== */

WITH Training_Data AS (
    SELECT 
        DeptID,
        
        -- Evaluate timely completion
        CASE 
            WHEN DueDate IS NULL OR TRIM(DueDate) = '' THEN 0
            WHEN CompletionDateUTC IS NULL OR TRIM(CompletionDateUTC) = '' THEN 0
            WHEN CAST(CompletionDateUTC AS DATE) <= CAST(DueDate AS DATE) THEN 1
            ELSE 0 
        END AS Completed_On_Time
        
    FROM hive_metastore.ra_adido_2025.td_code_of_conduct_and_ethics_11012024_10312025
)

SELECT 
    m.`Assessable Unit ID and Name` AS `Assessable Unit`,
    
    COUNT(t.DeptID) AS `Total Employees`,
    SUM(t.Completed_On_Time) AS `Total Completed On Time`,
    
    -- Calculate percentage, preventing division by zero
    ROUND(
        (SUM(t.Completed_On_Time) * 100.0) / NULLIF(COUNT(t.DeptID), 0), 
        2
    ) AS `Timely Completion Percentage`

FROM cost_center m

LEFT JOIN Training_Data t
    -- Smart Join: Adds a leading '0' only if the Mapping ID is exactly 3 characters
    ON CASE 
        WHEN LENGTH(TRIM(CAST(m.`Cost Center` AS STRING))) = 3 
        THEN CONCAT('0', TRIM(CAST(m.`Cost Center` AS STRING))) 
        ELSE TRIM(CAST(m.`Cost Center` AS STRING)) 
       END = TRIM(t.DeptID)

WHERE m.`Assessable Unit ID and Name` IS NOT NULL

GROUP BY 
    m.`Assessable Unit ID and Name`